In [ ]:
# Notebook de démonstration - Learning Assistant RAG
# Ce notebook montre comment utiliser le système programmatiquement

# ========================================
# Cellule 1: Setup et imports
# ========================================
import sys
from pathlib import Path
sys.path.append('../src')

from modules.ingestion import DocumentIngestion
from modules.chunking import TextChunker
from modules.embeddings import EmbeddingModel
from modules.retrieval import FAISSRetriever
from modules.learning_generator import LearningResponseGenerator
from modules.learning_config import learning_config
from modules.config import config

print("✅ Tous les modules importés")

# ========================================
# Cellule 2: Initialiser les composants
# ========================================
print("🚀 Initialisation du Learning Assistant...")

ingestion = DocumentIngestion()
chunker = TextChunker()
retriever = FAISSRetriever()
generator = LearningResponseGenerator()

print("✅ Composants initialisés")
print(f"📊 Modèle d'embedding : {retriever.embedding_model.model_name}")
print(f"🤖 Modèle de génération : {generator.model_name}")

# ========================================
# Cellule 3: Créer un cours d'exemple
# ========================================
sample_course = """
COURS : INTRODUCTION AU MACHINE LEARNING

Chapitre 1: Qu'est-ce que le Machine Learning ?

Le Machine Learning (apprentissage automatique) est une branche de l'intelligence 
artificielle qui permet aux ordinateurs d'apprendre à partir de données sans être 
explicitement programmés. Il existe trois types principaux :

1. Apprentissage Supervisé
Dans l'apprentissage supervisé, le modèle apprend à partir de données étiquetées.
On fournit au modèle des exemples avec leurs réponses correctes.

Exemples d'algorithmes supervisés :
- Régression linéaire : pour prédire des valeurs continues
- Arbres de décision : pour la classification
- Réseaux de neurones : pour des tâches complexes

2. Apprentissage Non-Supervisé
L'apprentissage non-supervisé travaille avec des données non étiquetées.
Le modèle cherche à découvrir des structures cachées dans les données.

Exemples :
- Clustering (K-means) : regrouper des données similaires
- Réduction de dimensionnalité (PCA) : simplifier les données

3. Apprentissage par Renforcement
Le modèle apprend en interagissant avec un environnement et en recevant
des récompenses ou des pénalités.

Chapitre 2: Les Réseaux de Neurones

Un réseau de neurones est inspiré du cerveau humain. Il est composé de :
- Couche d'entrée : reçoit les données
- Couches cachées : traite l'information
- Couche de sortie : produit le résultat

Le processus d'apprentissage utilise la rétropropagation (backpropagation).

Chapitre 3: Gradient Descent

Le gradient descent est un algorithme d'optimisation. Il fonctionne en suivant
les étapes suivantes :

1. Initialiser les paramètres du modèle
2. Calculer l'erreur (loss function)
3. Calculer le gradient (direction de la descente)
4. Mettre à jour les paramètres
5. Répéter jusqu'à convergence

Il existe plusieurs variantes :
- Batch Gradient Descent
- Stochastic Gradient Descent (SGD)
- Mini-batch Gradient Descent
"""

# Sauvegarder le cours
course_path = config.DOCUMENTS_DIR / "cours_ml.txt"
with open(course_path, 'w', encoding='utf-8') as f:
    f.write(sample_course)

print(f"✅ Cours d'exemple créé : {course_path}")

# ========================================
# Cellule 4: Indexer le cours
# ========================================
print("\n📚 Indexation du cours...")

# Extraction
doc_info = ingestion.process_document(course_path)
print(f"✅ Texte extrait : {doc_info['num_characters']} caractères")

# Chunking
chunks = chunker.create_chunks_with_metadata(
    doc_info['content'],
    doc_info['filename']
)
print(f"✅ {len(chunks)} chunks créés")

# Embeddings
chunk_texts = [c['content'] for c in chunks]
embeddings = retriever.embedding_model.encode(chunk_texts)
print(f"✅ Embeddings générés : {embeddings.shape}")

# Indexation
retriever.create_index(embeddings, chunks)
retriever.save_index()
print(f"✅ Index FAISS créé et sauvegardé")

# ========================================
# Cellule 5: Fonction de test pédagogique
# ========================================
def ask_learning_question(
    question: str, 
    level: str = 'intermediate',
    verbose: bool = True
):
    """
    Pose une question au Learning Assistant
    
    Args:
        question: La question
        level: beginner, intermediate, advanced
        verbose: Afficher les détails
    """
    if verbose:
        print(f"\n{'='*80}")
        print(f"🎓 Niveau : {level.upper()}")
        print(f"❓ Question : {question}")
        print(f"{'='*80}\n")
    
    # Détecter le type de question
    q_type = learning_config.detect_question_type(question)
    if verbose:
        print(f"📑 Type de question détecté : {q_type}\n")
    
    # Rechercher
    chunks = retriever.search(question, top_k=5)
    if verbose:
        print(f"🔍 {len(chunks)} passages récupérés\n")
    
    # Générer la réponse
    result = generator.generate_pedagogical_answer(
        question=question,
        context_chunks=chunks,
        learning_level=level
    )
    
    # Afficher
    if verbose:
        print(f"💡 RÉPONSE :")
        print(f"{'-'*80}")
        print(result['answer'])
        print(f"{'-'*80}\n")
        
        print(f"📚 Sources utilisées : {result['context_used']}")
        
        if result.get('follow_up_suggestions'):
            print(f"\n💭 Questions de suivi suggérées :")
            for i, sugg in enumerate(result['follow_up_suggestions'], 1):
                print(f"   {i}. {sugg}")
    
    return result

# ========================================
# Cellule 6: Test niveau DÉBUTANT
# ========================================
print("\n" + "="*80)
print("TEST 1 : NIVEAU DÉBUTANT")
print("="*80)

result1 = ask_learning_question(
    "C'est quoi le machine learning ?",
    level='beginner'
)

# ========================================
# Cellule 7: Test niveau INTERMÉDIAIRE
# ========================================
print("\n" + "="*80)
print("TEST 2 : NIVEAU INTERMÉDIAIRE")
print("="*80)

result2 = ask_learning_question(
    "Comment fonctionne un réseau de neurones ?",
    level='intermediate'
)

# ========================================
# Cellule 8: Test niveau AVANCÉ
# ========================================
print("\n" + "="*80)
print("TEST 3 : NIVEAU AVANCÉ")
print("="*80)

result3 = ask_learning_question(
    "Explique le processus de gradient descent en détail",
    level='advanced'
)

# ========================================
# Cellule 9: Test COMPARAISON
# ========================================
print("\n" + "="*80)
print("TEST 4 : COMPARAISON")
print("="*80)

result4 = ask_learning_question(
    "Quelle est la différence entre apprentissage supervisé et non supervisé ?",
    level='intermediate'
)

# ========================================
# Cellule 10: Test EXEMPLE
# ========================================
print("\n" + "="*80)
print("TEST 5 : DEMANDE D'EXEMPLES")
print("="*80)

result5 = ask_learning_question(
    "Donne-moi des exemples d'algorithmes supervisés",
    level='beginner'
)

# ========================================
# Cellule 11: Statistiques du système
# ========================================
print("\n" + "="*80)
print("📊 STATISTIQUES DU SYSTÈME")
print("="*80)

print(f"\n📚 Cours indexés :")
print(f"   - Nombre total de chunks : {retriever.index.ntotal}")
print(f"   - Dimension des vecteurs : {retriever.dimension}")

from collections import Counter
doc_counts = Counter(c['document_name'] for c in retriever.metadata)
print(f"\n📄 Répartition :")
for doc, count in doc_counts.items():
    print(f"   - {doc} : {count} chunks")

# ========================================
# Cellule 12: Analyser les types de questions
# ========================================
test_questions = [
    "C'est quoi le deep learning ?",
    "Comment fonctionne la backpropagation ?",
    "Donne-moi un exemple de clustering",
    "Quelle est la différence entre CNN et RNN ?",
    "Quelles sont les étapes pour entraîner un modèle ?",
    "Explique pourquoi on utilise la normalisation"
]

print("\n" + "="*80)
print("🔍 ANALYSE DES TYPES DE QUESTIONS")
print("="*80 + "\n")

for question in test_questions:
    q_type = learning_config.detect_question_type(question)
    print(f"❓ {question}")
    print(f"   → Type : {q_type}\n")

# ========================================
# Cellule 13: Export des résultats
# ========================================
import json
from datetime import datetime

# Préparer les résultats
test_results = {
    'timestamp': datetime.now().isoformat(),
    'system_info': {
        'embedding_model': retriever.embedding_model.model_name,
        'llm_model': generator.model_name,
        'total_chunks': retriever.index.ntotal
    },
    'tests': [
        {
            'question': result1.get('answer', '')[:100] + '...',
            'level': 'beginner',
            'type': result1.get('question_type', 'N/A')
        },
        {
            'question': result2.get('answer', '')[:100] + '...',
            'level': 'intermediate',
            'type': result2.get('question_type', 'N/A')
        },
        {
            'question': result3.get('answer', '')[:100] + '...',
            'level': 'advanced',
            'type': result3.get('question_type', 'N/A')
        }
    ]
}

# Sauvegarder
output_path = config.DATA_DIR / f"test_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
with open(output_path, 'w', encoding='utf-8') as f:
    json.dump(test_results, f, ensure_ascii=False, indent=2)

print(f"✅ Résultats exportés : {output_path}")

print("\n" + "="*80)
print("✅ DÉMO TERMINÉE")
print("="*80)